This notebook is used to create two ML models. 

One is used for predicting None values in Absorbance column.

One is used for predicting None values in PL max column.

Three algorithms are considered: Bagging, Decision Trees, and Random Forests

In [1]:
import numpy as np
import pandas as pd
import os
import joblib
import sklearn         
from sklearn import linear_model, datasets
from sklearn.utils import resample
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.svm import SVR
from tqdm import tqdm

# 1. Model for Absorbance 

In [2]:
# Absorbance
df_ab = pd.read_csv('dataset_CdSe_abs_filler.csv')

In [3]:
# This dataset excludes all rows that have 'None' in the 'Absorbance max (nm)' column.
# This dataset is used to train and create a model for predicting absorbance.
df_ab

,Growth Temp (Celsius),Metal_mmol (mmol),Chalcogen_mmol (mmol),CA_mmol (mmol),Amines_mmol (mmol),Phosphines_mmol (mmol),S_I_amount (g),S_II_amount (g),Time_min (min),x0_cadmium acetate,...,x4_liquid parafin,x4_octadecene,x4_phenyl ether,x4_trioctylphosphine oxide,x5_None,x5_phosphinic acid,x5_trioctylphosphine oxide,diameter_nm,abs_nm,emission_nm
0,1.102480,-0.438782,-0.245227,-0.525726,-0.417578,-0.166760,-0.124145,-0.302794,-0.226629,0,...,0,1,0,0,1,0,0,3.41,566,575
1,0.403387,-0.373263,-0.551908,0.128152,-0.499463,-0.265753,-0.681471,-0.302794,-0.236682,0,...,0,0,0,0,1,0,0,2.60,526,556
2,0.403387,-0.373263,-0.551908,0.128152,-0.499463,-0.265753,-0.681471,-0.302794,-0.236328,0,...,0,0,0,0,1,0,0,3.20,559,580
3,0.403387,-0.373263,-0.551908,0.128152,-0.499463,-0.265753,-0.681471,-0.302794,-0.235621,0,...,0,0,0,0,1,0,0,3.60,574,598
4,0.403387,-0.373263,-0.551908,0.128152,-0.499463,-0.265753,-0.681471,-0.302794,-0.234955,0,...,0,0,0,0,1,0,0,3.90,582,601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.481064,-0.466369,-0.501557,-0.387095,-0.499463,-0.371514,-0.439424,5.968747,-0.236766,0,...,0,1,0,0,0,0,1,1.70,450,None
191,0.481064,-0.466369,-0.501557,-0.387095,-0.499463,-0.371514,-0.439424,5.968747,-0.233914,0,...,0,1,0,0,0,0,1,2.50,530,None
192,0.481064,-0.466369,-0.501557,-0.387095,-0.499463,-0.371514,-0.439424,5.968747,-0.207125,0,...,0,1,0,0,0,0,1,3.50,585,None
193,0.481064,-0.466369,-0.501557,-0.387095,-0.499463,-0.371514,-0.439424,5.968747,-0.181461,0,...,0,1,0,0,0,0,1,3.70,590,None


In [5]:
# Below is all the columns input
# Note: This includes diameter as a input to predict Abs max

input_col = ['Growth Temp (Celsius)', 'Metal_mmol (mmol)', 'Chalcogen_mmol (mmol)',
             'Amines_mmol (mmol)', 'CA_mmol (mmol)', 'Phosphines_mmol (mmol)', 
             'S_I_amount (g)', 'S_II_amount (g)', 'Time_min (min)', 
             'x0_cadmium acetate',  
             'x0_cadmium acetate dihydrate', 'x0_cadmium oxide', 
             'x0_cadmium stearate', 'x0_dimethylcadmium', 'x1_None', 
             'x1_benzoic acid', 'x1_dodecylphosphonic acid', 
             'x1_ethylphosphonic acid', 'x1_lauric acid', 
             'x1_myrstic acid', 'x1_oleic acid', 'x1_stearic acid',
             'x2_2-6-dimethylpyridine', 'x2_None', 'x2_aniline', 
             'x2_benzylamine', 'x2_dioctylamine/hexadecylamine',
             'x2_dodecylamine', 'x2_heptylamine', 'x2_hexadecylamine', 
             'x2_octadecylamine', 'x2_octylamine', 'x2_oleylamine', 
             'x2_pyridine', 'x2_trioctylamine', 'x3_None', 'x3_diphenylphosphine', 
             'x3_tributylphosphine', 'x3_trioctylphosphine', 
             'x3_triphenylphosphine', 'x4_None', 'x4_liquid parafin', 
             'x4_octadecene', 'x4_phenyl ether', 'x4_trioctylphosphine oxide', 
             'x5_None', 'x5_phosphinic acid', 'x5_trioctylphosphine oxide',
             'diameter_nm', ]

output_col_ab = ['abs_nm']

X_ab = df_ab[input_col]

Y_ab = df_ab[output_col_ab]

In [6]:
# Splitting dataset for training
X_train_ab, X_test_ab, Y_train_ab, Y_test_ab = train_test_split(X_ab, Y_ab, test_size=0.15, random_state=45, shuffle=True)

## 1a. Bagging

In [11]:
# This is a grid search for three parameters in the Bagging algorithm. 
# Parameters are: max_depth, n_estimators, random_state.
# This gives the best combination of the three parameters for the smallest mean squared error.

min_mse = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 21)):
    for j in range(1, 21):
        for k in range(5, 80, 5):
            
            B_regr_ab = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=i),
                                      n_estimators=j,
                                      random_state=k)
            
            B_regr_ab.fit(X_train_ab, np.ravel(Y_train_ab))
            
            B_Y_pred_ab = B_regr_ab.predict(X_test_ab)
            
            mse = mean_squared_error(Y_test_ab, B_Y_pred_ab)
            
            if (min_mse > mse):
                min_mse = mse
                min_i = i
                min_j = j
                min_k = k
            
print(min_mse, min_i, min_j, min_k)

100%|██████████| 20/20 [02:26<00:00,  7.31s/it]

122.07711908563559 11 13 30


## 1b. Decision Trees

In [10]:
# This is a grid search for three parameters in the Decision Trees algorithm. 
# Parameters are: max_depth, max_features, random_state.
# This gives the best combination of the three parameters for the smallest mean squared error.

min_mse = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 21)):
    for j in range(1, 21):
        for k in range(5, 80, 5):
            
            DT_regr_ab = DecisionTreeRegressor(max_depth=i,
                                max_features=j,
                                random_state=k)
            
            DT_regr_ab.fit(X_train_ab, Y_train_ab)

            DT_Y_pred_ab = DT_regr_ab.predict(X_test_ab)

            mse = mean_squared_error(Y_test_ab, DT_Y_pred_ab)
            
            if (min_mse > mse):
                min_mse = mse
                min_i = i
                min_j = j
                min_k = k
            
print(min_mse, min_i, min_j, min_k)

100%|██████████| 20/20 [00:32<00:00,  1.63s/it]

208.61680642706537 4 15 10


## 1c. Random Forests

In [8]:
# This is a grid search for three parameters in the Random Forest algorithm. 
# Parameters are: max_depth, n_estimators, max_features.
# Random_state is set to 45.
# This gives the best combination of the three parameters for the smallest mean squared error.

min_mse = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 21)):
    for j in range(1, 21):
        for k in range(2, 50, 2):
            RF_regr_ab = RandomForestRegressor(max_depth=i, 
                                            n_estimators=j, 
                                            max_features=k,
                                            random_state=45
                                                )
            RF_regr_ab.fit(X_train_ab, np.ravel(Y_train_ab))
            RF_Y_pred_ab = RF_regr_ab.predict(X_test_ab)

            mse = mean_squared_error(Y_test_ab, RF_Y_pred_ab)
            if (min_mse > mse):
                min_mse = mse
                min_i = i
                min_j = j
                min_k = k
            
print(min_mse, min_i, min_j, min_k)

100%|██████████| 20/20 [02:56<00:00,  8.83s/it]

107.32386903085589 6 5 22


In [12]:
RF_regr_ab = RandomForestRegressor(max_depth=6, 
                                            n_estimators=5, 
                                            max_features=22,
                                  random_state=45)
RF_regr_ab.fit(X_train_ab, np.ravel(Y_train_ab))
                                   
RF_Y_pred_ab = RF_regr_ab.predict(X_test_ab)

mse = mean_squared_error(Y_test_ab, RF_Y_pred_ab)
mae = mean_absolute_error(Y_test_ab, RF_Y_pred_ab)
                                   
print(mse,mae)

107.32386903085589 7.906736709527437


## Conclusion:

Random forest gives the least value of MSE with the parameters combination of: max_depth= **6**, n_estimators = **5**, max_features = **22**, and random_state = **45**.                  

## Saving model

In [24]:
RF_regr_ab = RandomForestRegressor(max_depth=6, 
                                n_estimators=5, 
                                max_features=22,
                                random_state=45
                                )
RF_regr_ab.fit(X_train_ab, np.ravel(Y_train_ab))
RF_Y_pred_ab = RF_regr_ab.predict(X_test_ab)

In [25]:
joblib.dump(RF_regr_ab, "./model_CdSe_RandomForest_abs_aug.joblib")

['./model_CdSe_RandomForest_abs_aug.joblib']

# 2. Model for Photoluminescence  (PL)

In [15]:
# PL
df_pl = pd.read_csv('dataset_CdSe_emission_filler.csv')

In [16]:
# This dataset excludes all rows that have 'None' in the 'PL max (nm)' column.
# This dataset is used to train and create a model for predicting absorbance.
df_pl

,Growth Temp (Celsius),Metal_mmol (mmol),Chalcogen_mmol (mmol),CA_mmol (mmol),Amines_mmol (mmol),Phosphines_mmol (mmol),S_I_amount (g),S_II_amount (g),Time_min (min),x0_cadmium acetate,...,x4_liquid parafin,x4_octadecene,x4_phenyl ether,x4_trioctylphosphine oxide,x5_None,x5_phosphinic acid,x5_trioctylphosphine oxide,diameter_nm,abs_nm,emission_nm
0,1.102480,-0.438782,-0.245227,-0.525726,-0.417578,-0.166760,-0.124145,-0.302794,-0.226629,0,...,0,1,0,0,1,0,0,3.41,566,575
1,0.325710,-0.473265,-0.588527,-0.525726,-0.499463,-0.452738,-0.527398,-0.302794,-0.235995,0,...,0,1,0,0,1,0,0,1.99,None,497
2,0.325710,-0.473265,-0.588527,-0.525726,-0.499463,-0.446393,-0.527398,-0.302794,-0.235995,0,...,0,1,0,0,1,0,0,2.13,None,510
3,0.325710,-0.473265,-0.588527,-0.525726,-0.499463,-0.446393,-0.527398,-0.302794,-0.234955,0,...,0,1,0,0,1,0,0,2.27,None,517
4,0.325710,-0.473265,-0.588527,-0.525726,-0.499463,-0.446393,-0.527398,-0.302794,-0.232873,0,...,0,1,0,0,1,0,0,2.53,None,522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,0.791772,-0.478093,-0.327619,-0.465653,-0.297091,0.021917,-0.464114,-0.302794,-0.226629,0,...,0,1,0,0,1,0,0,4.10,None,580
152,0.791772,-0.478093,-0.327619,-0.465653,0.015240,0.021917,-0.464114,-0.302794,-0.236932,0,...,0,1,0,0,1,0,0,3.70,None,535
153,0.791772,-0.478093,-0.327619,-0.465653,0.077238,0.021917,-0.464114,-0.302794,-0.236932,0,...,0,1,0,0,1,0,0,4.10,None,575
154,0.791772,-0.478093,-0.327619,-0.465653,-0.095889,0.021917,-0.464114,-0.302794,-0.236932,0,...,0,1,0,0,1,0,0,3.60,None,530


In [17]:
output_col_pl = ['emission_nm']

X_pl = df_pl[input_col]

Y_pl = df_pl[output_col_pl]

In [18]:
# Splitting dataset for training
X_train_pl, X_test_pl, Y_train_pl, Y_test_pl = train_test_split(X_pl, Y_pl, test_size=0.15, random_state=45, shuffle=True)

## 2a. Bagging


In [23]:
# This is a grid search for three parameters in the Bagging algorithm. 
# Parameters are: max_depth, n_estimators, random_state.
# This gives the best combination of the three parameters for the smallest mean squared error.

min_mse = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 21)):
    for j in range(1, 21):
        for k in range(5, 80, 5):
            
            B_regr_pl = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=i),
                                      n_estimators=j,
                                      random_state=k)
            
            B_regr_pl.fit(X_train_pl, np.ravel(Y_train_pl))
            
            B_Y_pred_pl = B_regr_pl.predict(X_test_pl)
            
            mse = mean_squared_error(Y_test_pl, B_Y_pred_pl)
            
            if (min_mse > mse):
                min_mse = mse
                min_i = i
                min_j = j
                min_k = k
            
print(min_mse, min_i, min_j, min_k)

100%|██████████| 20/20 [02:15<00:00,  6.79s/it]

253.45833333333334 9 1 10


## 2b. Decision Trees

In [32]:
# This is a grid search for three parameters in the Decision Trees algorithm. 
# Parameters are: max_depth, max_features, random_state.
# This gives the best combination of the three parameters for the smallest mean squared error.

min_mse = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 21)):
    for j in range(1, 21):
        for k in range(5, 80, 5):
            
            DT_regr_pl = DecisionTreeRegressor(max_depth=i,
                                max_features=j,
                                random_state=k)
            
            DT_regr_pl.fit(X_train_pl, Y_train_pl)

            DT_Y_pred_pl = DT_regr_pl.predict(X_test_pl)

            mse = mean_squared_error(Y_test_pl, DT_Y_pred_pl)
            
            if (min_mse > mse):
                min_mse = mse
                min_i = i
                min_j = j
                min_k = k
            
print(min_mse, min_i, min_j, min_k)

100%|██████████| 20/20 [00:35<00:00,  1.77s/it]

208.33333333333334 12 13 45


## 2c Random Forest

In [20]:
# This is a grid search for three parameters in the Random Forest algorithm. 
# Parameters are: max_depth, n_estimators, max_features.
# Random_state is set to 45.
# This gives the best combination of the three parameters for the smallest mean squared error.

min_mse = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 21)):
    for j in range(1, 21):
        for k in range(2, 50, 2):
            RF_regr_pl = RandomForestRegressor(max_depth=i, 
                                            n_estimators=j, 
                                            max_features=k,
                                            random_state=45
                                                )
            RF_regr_pl.fit(X_train_pl, np.ravel(Y_train_pl))
            RF_Y_pred_pl = RF_regr_pl.predict(X_test_pl)

            mse = mean_squared_error(Y_test_pl, RF_Y_pred_pl)
            if (min_mse > mse):
                min_mse = mse
                min_i = i
                min_j = j
                min_k = k
            
print(min_mse, min_i, min_j, min_k)

100%|██████████| 20/20 [02:54<00:00,  8.72s/it]

177.49479166666666 11 4 8


In [22]:
RF_regr_pl = RandomForestRegressor(max_depth=11, 
                                            n_estimators=4, 
                                            max_features=8,
                                            random_state=45
                                                )
RF_regr_pl.fit(X_train_pl, np.ravel(Y_train_pl))
RF_Y_pred_pl = RF_regr_pl.predict(X_test_pl)

mse = mean_squared_error(Y_test_pl, RF_Y_pred_pl)
mae = mean_absolute_error(Y_test_pl, RF_Y_pred_pl)
print(mse, mae)

177.49479166666666 10.3125


## Conclusion 

Random forest gives the least value of MSE with the parameters combination of: max_depth= **12**, n_estimators = **7**, max_features = **8**, and random_state = **45**.        

## Saving model

In [26]:
RF_regr_pl = RandomForestRegressor(max_depth=11, 
                                n_estimators=4, 
                                max_features=8,
                                random_state=45
                                )
RF_regr_pl.fit(X_train_pl, np.ravel(Y_train_pl))
RF_Y_pred_pl = RF_regr_pl.predict(X_test_pl)

In [28]:
joblib.dump(RF_regr_pl, "./model_CdSe_RandomForest_emission_aug.joblib")

['./model_CdSe_RandomForest_emission_aug.joblib']